# 197. Rising temperature
[Problem description](https://leetcode.com/problems/rising-temperature/description/)

In [1]:
import sys
sys.version

'3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]'

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]')\
    .appName('LeetCode-197-PySpark')\
    .getOrCreate()

In [10]:
# Data 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import to_date, col
schema = StructType([
    StructField('id', IntegerType(), False),
    StructField('date', StringType(), False), # later parse to recordDate: Date
    StructField('temperature', IntegerType(), False),
])

weather = spark.createDataFrame([[1, '2015-01-01', 10], [2, '2015-01-02', 25], [3, '2015-01-03', 20], [4, '2015-01-04', 30]], schema=schema)
weather = weather.withColumn('recordDate', to_date(col('date'), 'yyyy-MM-dd')).drop('date')
weather.show()

+---+-----------+----------+
| id|temperature|recordDate|
+---+-----------+----------+
|  1|         10|2015-01-01|
|  2|         25|2015-01-02|
|  3|         20|2015-01-03|
|  4|         30|2015-01-04|
+---+-----------+----------+



**Write a solution to find all dates' Id with higher temperatures compared to its previous dates (yesterday).**

In [25]:
# Solution
from pyspark.sql import Window
from pyspark.sql.functions import lag, col, lit

df = weather.withColumn('group', lit(1))
window = Window.partitionBy('group').orderBy('id')

df = df.withColumn('rising', col('temperature') > lag(col('temperature')).over(window)).drop('group')
df.filter(df.rising == 'true').drop('temperature', 'recordDate', 'rising').show()

+---+
| id|
+---+
|  2|
|  4|
+---+

